In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
def convert_loc(row):  #convert longlat to meter/km
    splitted = row['loc'].split(',')
    row['lon'] = splitted[0]
    row['lat'] = splitted[1]
    return row
def convert_speed(row):  #convert longlat to meter/km
    splitted = row['v'].split(',')
    row['vx'] = splitted[0]
    row['vy'] = splitted[1]
    return row
def join_cols(row):
    row['loc'] = str(row['lon'])+','+str(row['lat'])
    row['v'] = str(row['vx'])+','+str(row['vy'])
    return row
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

In [20]:
infile = "events_Approach - Bypass1000.txt"
df = pd.read_csv(infile, sep='|', names=['t', 'id', 'loc', 'v'])
# df = df.apply(lambda row: convert_loc(row), axis=1)
# df = df.apply(lambda row: convert_speed(row), axis=1)
# df['vx'] = df['vx'].astype(float)
# df['vy'] = df['vy'].astype(float)
# df['lon'] = df['lon'].astype(float)
# df['lat'] = df['lat'].astype(float)
df.head()

,t,id,loc,v
0,0,0,"435909,520893","-192.233,-35.3033"
1,0,1,"639659,945893","83.0765,27.2055"
2,0,2,"540909,563393","163.625,-64.405"
3,0,3,"203409,1.01214e+06","-2.0124,-24.5743"
4,0,4,"300909,632143","-89.2487,-118.45"


In [19]:
df.t.max()

199

In [21]:
# MIN_vx = df.vx.min()
# MAX_vx = df.vx.max()
# MIN_vy = df.vy.min()
# MAX_vy = df.vy.max()
# MIN_lon = df.lon.min()
# MAX_lon = df.lon.max()
# MIN_lat = df.lat.min()
# MAX_lat = df.lat.max()
MIN_vx = -220.48
MAX_vx = 919.433
MIN_vy = -622.385
MAX_vy = 347.639
MIN_lon = -159091
MAX_lon = 748657
MIN_lat = 207143
MAX_lat = 1077140
lon_mean = 327933.06507
lon_std = 268541.72758
lat_mean = 642868.54999
lat_std = 243375.45802
print("lon mean&std is "+str(lon_mean)+" to " +str(lon_std))
print("lat mean&std is "+str(lat_mean)+" to " +str(lat_std))
df.info()

lon mean&std is 327933.06507 to 268541.72758
lat mean&std is 642868.54999 to 243375.45802
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   t       200000 non-null  int64 
 1   id      200000 non-null  int64 
 2   loc     200000 non-null  object
 3   v       200000 non-null  object
dtypes: int64(2), object(2)
memory usage: 6.1+ MB


In [22]:
vesselNum = int(df.id.max()/2)
rMin = 300
rMax = 500
data = []
i = 0
while i < vesselNum:
    vesID = i
    vx = random.uniform(MIN_vx, MAX_vx)
    vy = random.uniform(MIN_vy, MAX_vy)
    r = random.randrange(rMin, rMax)
    data.append([vesID, vx, vy, r])
    i += 1
    
result = pd.DataFrame(data, columns = ['id','vx', 'vy', 'r'])
result['lon'] = trunc(np.random.normal(lon_mean, lon_std, vesselNum), 4)
result['lat'] = trunc(np.random.normal(lat_mean, lat_std, vesselNum), 4)
result['vx'] = trunc(result['vx'], 4)
result['vy'] = trunc(result['vy'], 4)
result.head()

,id,vx,vy,r,lon,lat
0,0,236.6760,187.3267,362,55642.4565,655602.1580
1,1,538.3053,-572.3759,364,243196.4824,729136.0059
2,2,833.1571,109.9938,466,147531.5568,483841.7838
3,3,837.6722,-45.8719,336,116093.5041,335733.9667
4,4,605.4281,-373.9092,461,56988.1644,-10711.0136


In [23]:
result = result.apply(lambda row: join_cols(row), axis=1)
del result['vx']
del result['vy']
del result['lon']
del result['lat']
result['id'] = result['id'].astype(int)
neworder = ['id','loc','v', 'r']
result=result.reindex(columns=neworder)
result.head()

,id,loc,v,r
0,0,"55642.4565,655602.158","236.676,187.3267",362.0
1,1,"243196.4824,729136.0059","538.3053,-572.3759",364.0
2,2,"147531.5568,483841.7838","833.1571,109.9938",466.0
3,3,"116093.5041,335733.9667","837.6722,-45.8719",336.0
4,4,"56988.1644,-10711.0136","605.4281,-373.9092",461.0


In [24]:
filename = 'vessel_'+str(len(result))+'.csv'
result.to_csv(filename, sep='|', header=False, index=False)